In [11]:
!pip install googlesearch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement googlesearch (from versions: none)
ERROR: No matching distribution found for googlesearch


In [14]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
import json
#from googlesearch import search  # For web search in CRAG

# Load IPL dataset
ipl_data = pd.read_csv("matches.csv")

month = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]
months = {}

for i in range(len(month)):
    months[i+1] = month[i]
# print(months)

from sentence_transformers import SentenceTransformer

# Load pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2') 

# Combine dataset columns for retrieval
ipl_data['context'] = ipl_data.apply(
    lambda x: f"Season: {x['season']}, Date: {x['date']} or {x['date'].split('/')[0]} of {months[int(x['date'].split('/')[1])]} {x['date'].split('/')[2]}, City: {x['city']}, Match: {x['team1']} vs {x['team2']}, "
              f"Toss Winner: {x['toss_winner']}, Decision: {x['toss_decision']}, Result: {x['result']}, Duckworth-Lewis-Stern applied: {x['dl_applied']}, "
              f"Winner: {x['winner']}, Win By Runs: {x['win_by_runs']}, Win By Wickets: {x['win_by_wickets']}, Player of the Match: {x['player_of_match']}, Venue: {x['venue']}, Umpire1: {x['umpire1']}, Umpire2: {x['umpire2']}",
    axis=1
)

# TF-IDF vectorization for retrieval
# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(ipl_data['context'])
tfidf_matrix = model.encode(ipl_data['context'])

# Function to retrieve relevant context
def retrieve_context(question, k=5):
    question_vec = model.encode([question])
    scores = cosine_similarity(question_vec, tfidf_matrix).flatten()
    top_indices = scores.argsort()[-k:][::-1]
    return ipl_data.iloc[top_indices]['context'].tolist()

# Function to query Phi3 via Ollama API
def query_phi3(prompt):
    url = "http://localhost:11434/api/generate"  # Replace with your server URL if using ngrok
    headers = {"Content-Type": "application/json"}
    payload = {"model": "phi3", "prompt": prompt}
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        return "Error querying Phi3 model."

def query_phi3(prompt):
    url = "http://localhost:11434/api/generate/"  # Replace with ngrok URL if applicable
    headers = {"Content-Type": "application/json"}
    payload = {"model": "phi3", "prompt": prompt}

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # Raise an error for bad responses
        return response.json().get("response", "No response from Phi3 model.")
    except requests.exceptions.RequestException as e:
        return f"Error querying Phi3 model: {e}"

def query_phi3(prompt):
    url = "http://localhost:11434/api/generate"  # Ensure this is the correct endpoint
    headers = {"Content-Type": "application/json"}
    payload = {"model": "phi3", "prompt": prompt}

    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()  # Handle HTTP errors
        resp = "["
        # print("Raw Response:", response.text) # Log the raw response for debugging)
        for i in response:
            resp += i.decode('utf-8').replace("\n", "").replace("}", "},")
        resp = resp[:-1] + "]"
        resp = json.loads(resp)
        ans = ""
        for i in resp:
            ans += i["response"]
        return ans
    except requests.exceptions.RequestException as e:
        return f"Error querying Phi3 model: {e}"
    except ValueError as e:
        return f"Error parsing JSON response: {e}"


# Function to perform a web search
def web_search(question, num_results=3):
    results = search(question, num_results=num_results)
    return "\n".join(results)

# RAG system function
def answer_question_rag(question):
    # Retrieve context from the dataset
    context = retrieve_context(question)
    print("The context retrieved is: ", context)
    if context:
        full_context = "\n".join(context)
        prompt = f"Using the following IPL dataset context, answer the question:\n\n{full_context}\n\nQuestion: {question}\nAnswer:"
        response = query_phi3(prompt)
        if response.strip().lower() not in ["i don't know", "not found", ""]:
            return response
    return None

# CRAG system function (with web search fallback)
def answer_question_crag(question):
    # Try to answer using the RAG system
    rag_response = answer_question_rag(question)
    if rag_response:
        return rag_response
    
    # If RAG fails, use web search and ask Phi3
    web_results = web_search(question)
    if web_results:
        prompt = f"Using the following web search results, answer the question:\n\n{web_results}\n\nQuestion: {question}\nAnswer:"
        return query_phi3(prompt)
    
    return "I'm sorry, I couldn't find the answer to your question."

question = "In what locations did KKR and CSK play a match in the year 2010 ?"

# RAG system response
print("Using RAG system...")
rag_answer = answer_question_rag(question)
print("RAG Answer:", rag_answer if rag_answer else "No answer found in the dataset.")

# CRAG system response
# print("\nUsing CRAG system...")
# crag_answer = answer_question_crag(question)
# print("CRAG Answer:", crag_answer)


/opt/anaconda3/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Using RAG system...
The context retrieved is:  ['Season: 2010, Date: 4/4/2010 or 4 of april 2010, City: Kolkata, Match: Kolkata Knight Riders vs Kings XI Punjab, Toss Winner: Kolkata Knight Riders, Decision: bat, Result: normal, Duckworth-Lewis-Stern applied: 0, Winner: Kings XI Punjab, Win By Runs: 0, Win By Wickets: 8, Player of the Match: DPMD Jayawardene, Venue: Eden Gardens, Umpire1: S Asnani, Umpire2: DJ Harper', 'Season: 2010, Date: 21/03/10 or 21 of march 10, City: Chennai, Match: Kings XI Punjab vs Chennai Super Kings, Toss Winner: Chennai Super Kings, Decision: field, Result: tie, Duckworth-Lewis-Stern applied: 0, Winner: Kings XI Punjab, Win By Runs: 0, Win By Wickets: 0, Player of the Match: J Theron, Venue: MA Chidambaram Stadium, Chepauk, Umpire1: K Hariharan, Umpire2: DJ Harper', 'Season: 2010, Date: 27/03/10 or 27 of march 10, City: Chandigarh, Match: Kolkata Knight Riders vs Kings XI Punjab, Toss Winner: Kolkata Knight Riders, Decision: bat, Result: normal, Duckworth-L

In [4]:
!ps aux | grep ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tsrk04           24757  33.6  0.0 34403536   3260 s011  Ss+  12:18PM   0:00.24 /bin/zsh -c ps aux | grep ollama
tsrk04           24760   0.0  0.0 33734900   1972 s011  U+   12:18PM   0:00.00 grep ollama


In [5]:
!ollama serve

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2024/12/30 12:19:00 routes.go:1259: INFO server config env="map[HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/Users/tsrk04/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPREAD:false http_proxy: https_proxy: no_proxy:]"
time=2024-12-30T12:19:00.750+05:30 level=INFO source=images.go:757 msg="total blobs: 5"
time=2024-12-30T12:19:00.750+05:30 level=INFO source=images.go:764 msg="total unused blobs r

In [6]:
!ollama list

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NAME           ID              SIZE      MODIFIED     
phi3:latest    4f2222927938    2.2 GB    17 hours ago    


In [8]:
!curl http://localhost:11434/api/generate -d '{"model": "phi3", "prompt": "Hello"}'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


{"model":"phi3","created_at":"2024-12-30T06:50:49.796865Z","response":"Hello","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:49.853353Z","response":"!","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:49.908247Z","response":" How","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:49.960808Z","response":" can","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:50.014812Z","response":" I","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:50.090084Z","response":" help","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:50.153812Z","response":" you","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:50.221612Z","response":" today","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:50.290942Z","response":"?","done":false}
{"model":"phi3","created_at":"2024-12-30T06:50:50.367752Z","response":"","done":true,"done_reason":"stop","context":[32010,29871,13,10994,32007,29871,13,32001,29871,13,10994,29991,1128,